## Imports and Constants

In [60]:
import uproot
import matplotlib.pyplot as plt
import numpy as np
import time
from IPython.display import display, clear_output
import pandas as pd

pd.set_option('display.max_columns', 500)

In [61]:
outdir= './output/DaughterTraining/'
inputdir = './input/'  

nu_file = inputdir+'flashID_nu_mcc9_nov_200k.root'
nue_file = inputdir+'flashID_nue_mcc9_nov_50k.root'

In [62]:
lower = [-1.55, -115.53, 0.1]
upper = [254.8, 117.47, 1036.9]
fidvol_vtx = [10,10,10,10,10,50] 
fidvol_end = [10,10,10,10,10,10]

## Functions

In [63]:
def inTPC_df(df, str_x, str_y, str_z, fidvol=[0]*6):
    global upper, lower
    mask_x = df[str_x].between(lower[0]+fidvol[0], upper[0]-fidvol[1])
    mask_y = df[str_y].between(lower[1]+fidvol[2], upper[1]-fidvol[3])
    mask_z = df[str_z].between(lower[2]+fidvol[4], upper[2]-fidvol[5])
    mask = mask_x & mask_y & mask_z
    return mask

## Load data

In [64]:
nu_daughters = uproot.open(nu_file)['nueCCAnalyser/Daughters'].pandas.df()
nue_daughters = uproot.open(nue_file)['nueCCAnalyser/Daughters'].pandas.df()

In [65]:
daughters = pd.concat([nu_daughters,nue_daughters])

In [66]:
list(daughters.columns)

['event',
 'run',
 'subrun',
 'evt_time_sec',
 'evt_time_nsec',
 'hitsU',
 'hitsV',
 'hitsY',
 'caloU',
 'caloV',
 'caloY',
 'hitsSps',
 'generation',
 'track_score',
 'is_shower',
 'is_track',
 'has_shower_daughter',
 'is_track_daughter',
 'vx',
 'vy',
 'vz',
 'vtx_distance',
 'track_length',
 'track_endx',
 'track_endy',
 'track_endz',
 'track_dirx',
 'track_diry',
 'track_dirz',
 'shower_length',
 'shower_openangle',
 'shower_dirx',
 'shower_diry',
 'shower_dirz',
 'start_dedxU',
 'start_dedxV',
 'start_dedxY',
 'start_hitsU',
 'start_hitsV',
 'start_hitsY',
 'start_pitchU',
 'start_pitchV',
 'start_pitchY',
 'mc_neutrino',
 'mc_vx',
 'mc_vy',
 'mc_vz',
 'mc_vx_sce',
 'mc_vy_sce',
 'mc_vz_sce',
 'mc_energy',
 'mc_pdg']

## Add features

In [67]:
# Add reco containment
start_mask = inTPC_df(daughters, 'vx', 'vy', 'vz', fidvol_vtx)
end_mask = inTPC_df(daughters, 'track_endx', 'track_endy', 'track_endz', fidvol_vtx)
daughters['contained'] = start_mask & end_mask 

# Add theta angle
v1_u = daughters[['track_dirx','track_diry','track_dirz']]
v2_u = [0,0,1]
daughters['track_theta'] = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
v3_u = [0,-1,0]
daughters['track_zenith'] = np.arccos(np.clip(np.dot(v1_u, v3_u), -1.0, 1.0))

# Add true fidvol
daughters['mc_fidvol'] = inTPC_df(daughters, 'mc_vx', 'mc_vy', 'mc_vz', fidvol_vtx)

# Add distance from true to reco vertex: 
eval_str = "sqrt( (vx-mc_vx_sce)**2 + (vy-mc_vy_sce)**2 + (vz-mc_vz_sce)**2 )"
daughters['mc_reco_vtx_distance'] = daughters.eval(eval_str)

# Add categories to train/test on
good_str = "mc_reco_vtx_distance<5 & mc_neutrino & mc_fidvol & "
cat_e_good = good_str+" abs(mc_pdg)==11"
cat_p_good = good_str+" abs(mc_pdg)==2212"
cat_g_good = "mc_reco_vtx_distance<75 & mc_neutrino & mc_fidvol &  mc_pdg==22"
cat_mu_good = good_str+" abs(mc_pdg)==13"
cat_pi_good = good_str+" abs(mc_pdg)==211"
cat_cosmic = "~mc_neutrino"
daughters['label_train'] =   1*daughters.eval(cat_e_good)+\
                             2*daughters.eval(cat_p_good)+\
                             3*daughters.eval(cat_g_good)+\
                             4*daughters.eval(cat_mu_good)+\
                             5*daughters.eval(cat_pi_good)+\
                             6*daughters.eval(cat_cosmic)

daughters['label_test'] =    1*daughters.eval("abs(mc_pdg)==11")+\
                             2*daughters.eval("abs(mc_pdg)==2212")+\
                             3*daughters.eval("mc_pdg==22")+\
                             4*daughters.eval("mc_neutrino & abs(mc_pdg)==13")+\
                             5*daughters.eval("abs(mc_pdg)==211")+\
                             6*daughters.eval("~mc_neutrino & abs(mc_pdg)==13")

In [68]:
daughters['label_test'].value_counts()

2    83607
4    74473
1    57752
6    45850
5    35080
3    23009
0     8249
Name: label_test, dtype: int64

## Classify

In [69]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from joblib import dump

In [70]:
train_cols = ['caloU', 'caloV', 'caloY', 'hitsSps', 'generation', 'track_score', 
              'has_shower_daughter', 'is_track_daughter',
              'vtx_distance','track_length','shower_openangle',
              'start_dedxU', 'start_dedxV', 'start_dedxY', 
              'start_hitsU', 'start_hitsV', 'start_hitsY',
              'start_pitchU', 'start_pitchV', 'start_pitchY',
              'track_theta', 'track_zenith']
label_col = 'label_train'
seed = 7
test_size = 0.2

In [71]:
X = daughters.query("contained & label_train>0")[train_cols]
Y = daughters.query("contained & label_train>0")[label_col].ravel()

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [73]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [85]:
# dump model:
dump(model, outdir+'xgb_daughter_classifier.joblib') 

['./output/DaughterTraining/xgb_daughter_classifier.joblib']

In [75]:
# make predictions for test data
y_pred = model.predict(X_test)
target_names = ['electron', 'proton', 'photon', 'nu muon', 'charged pion', 'cosmic']
print(classification_report(y_test, y_pred, target_names=target_names))
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Test accuracy: %.2f%%" % (accuracy * 100.0))


# make predictions for test data
y_pred = model.predict(X_train)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_train, predictions)
print("Train accuracy: %.2f%%" % (accuracy * 100.0))

              precision    recall  f1-score   support

    electron       0.84      0.76      0.80      2799
      proton       0.80      0.94      0.87      9544
      photon       0.76      0.68      0.72      2760
     nu muon       0.70      0.62      0.65      2689
charged pion       0.47      0.27      0.35      2105
      cosmic       0.72      0.74      0.73      5821

   micro avg       0.76      0.76      0.76     25718
   macro avg       0.72      0.67      0.68     25718
weighted avg       0.75      0.76      0.75     25718

Test accuracy: 75.74%
Train accuracy: 75.93%


In [82]:
importances = model.feature_importances_
sort=np.argsort(-importances)

for i,(n,im) in enumerate(zip(np.array(train_cols)[sort],importances[sort])):
    print("%d. feature %s (%f)" % (i + 1, n, im))

1. feature track_score (0.137956)
2. feature vtx_distance (0.122230)
3. feature shower_openangle (0.083393)
4. feature track_length (0.082678)
5. feature start_dedxY (0.080057)
6. feature hitsSps (0.067906)
7. feature track_theta (0.059566)
8. feature caloY (0.056707)
9. feature caloV (0.052418)
10. feature start_dedxV (0.043841)
11. feature start_dedxU (0.042173)
12. feature has_shower_daughter (0.037884)
13. feature is_track_daughter (0.030021)
14. feature caloU (0.019538)
15. feature track_zenith (0.019061)
16. feature start_pitchV (0.018108)
17. feature start_pitchU (0.015249)
18. feature start_hitsY (0.011675)
19. feature start_pitchY (0.007863)
20. feature start_hitsV (0.005957)
21. feature start_hitsU (0.003812)
22. feature generation (0.001906)


In [49]:
daughters['xgb'] = model.predict(daughters[train_cols])
label_map = dict(zip(range(1,7),target_names))
daughters['xgb_label'] = daughters['xgb'].map(label_map)

In [50]:
daughters[['event', 'mc_neutrino', 'mc_pdg','xgb_label']].tail(50)

,event,mc_neutrino,mc_pdg,xgb_label
126739,12462,True,2212,photon
126740,12462,True,2212,photon
126741,12462,True,2212,proton
126742,12462,False,-13,cosmic
126743,12462,False,-13,cosmic
126744,12464,False,2112,proton
126745,12465,False,13,cosmic
126746,12465,False,-13,cosmic
126747,12465,False,13,cosmic
126748,12465,False,-13,cosmic
